In [142]:
import sys 
sys.path.append('../')
import jax 
import jax.numpy as jnp 
import numpy as np
from jaxlie import SE3, SO3
from jax.random import normal
from jax import grad, hessian, vmap, pmap
from jax.flatten_util import ravel_pytree
from jax.lax import scan
from functools import partial
import jax.random as jax_random
import matplotlib.pyplot as plt 
from IPython.display import clear_output
from plyfile import PlyData
import open3d as o3d 
import trimesh as tm
import trimesh
import time
import jaxopt
import polyscope as ps


import meshcat
from meshcat.animation import Animation
import meshcat.geometry as mc_geom
import meshcat.transformations as mc_tf
from ergodic_mmd.aug_lagrange_jaxopt import AugmentedLagrangeSolver
# from ergodic_mmd.aug_lagrange_solver import AugmentedLagrangeSolver

import adam
from adam.jax import KinDynComputations

In [2]:
from robomeshcat import Object, Robot, Scene
from example_robot_data.robots_loader import PandaLoader
from pathlib import Path

"Create a scene that stores all objects and robots and has rendering capability"

scene = Scene()


robot = Robot(urdf_path=PandaLoader().df_path, mesh_folder_path=Path(PandaLoader().model_path).parent.parent)
scene.add_robot(robot)
"Render the initial scene"
scene.render()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [3]:

mesh = tm.primitives.Box([0.25, 0.25, 0.25])
# mesh.apply_scale(1.2)
# mesh.apply_transform(
#     SE3.from_rotation(SO3.from_x_radians(np.pi/2)).as_matrix()
#     )
mesh.apply_translation(np.array([0.5,0,0.125]))

num_points = 1000  # Change this number based on your requirement
points, face_indices = tm.sample.sample_surface(mesh, num_points)

info_distr = lambda x, n: jnp.maximum(0, jnp.dot(n, jnp.array([0,1,0])) + jnp.dot(n,jnp.array([0,0,1])))#jnp.exp(-20*(x[0]-0.5)**2 - 20*(x[1]+0.1)**2 - 20*(x[2]-0.4)**2)
P_XI = vmap(info_distr, in_axes=(0,0))(points, mesh.face_normals[face_indices])
P_XI_NORM = jnp.sum(P_XI)
P_XI = P_XI/P_XI_NORM
h = 0.01
args = {'h' : h, 
        'points' : points+0.05*mesh.face_normals[face_indices], 
        'normals' : mesh.face_normals[face_indices], 
        'P_XI' : P_XI}


# exp_obj = trimesh.exchange.obj.export_obj(mesh)
# mc_obj = mc_geom.ObjMeshGeometry.from_stream(trimesh.util.wrap_as_stream(exp_obj))
# mc_obj = Object(mc_obj, name='red_sphere', opacity=0.5, color=[1., 0., 0.])
# scene.add_object(mc_obj)

mesh_vertex_color_info = np.ones_like(mesh.vertices)
# _color_map = vmap(info_distr)(mesh.vertices)
# _color_map = _color_map - _color_map.min()
# # _color_map = _color_map/_color_map. max()
# mesh_vertex_color_info[:,0] = _color_map
# mc_obj = mc_geom.TriangularMeshGeometry(mesh.vertices, mesh.faces, color=mesh_vertex_color_info)
# scene.vis['obj'].set_object(
#     mc_obj, 
#     mc_geom.MeshPhongMaterial()
# )
box = meshcat.geometry.Box([0.25, 0.25, 0.25])
scene.vis['obj'].set_object(box, mc_geom.MeshLambertMaterial(color=0xADD8E6, reflectivity=0.8))
scene.vis['obj'].set_transform(mc_tf.translation_matrix([0.5,0,0.125]))
color_map = np.zeros_like(args['points'].T)
color_map[0,:] = P_XI*P_XI_NORM
scene.vis['obj_samples'].set_object(
        mc_geom.PointsGeometry(position=args['points'].T, color=color_map),
        mc_geom.PointsMaterial(size=0.01)
)

# mc_obj = Object(mc_obj, name='red_sphere', opacity=1)
# scene.add_object(mc_obj)

# verts = np.random.rand(3, 100000)
# scene.vis["perception/pointclouds/random"].set_object(
#     mc_geom.PointCloud(position=verts, color=verts))
# scene.vis["perception/pointclouds/random"].set_transform(
#     mc_tf.translation_matrix([0, 1, 0]))

In [4]:
model_path = "../assets/panda.urdf"
# The joint list
joints_name_list = [
    'panda_joint1', 'panda_joint2', 'panda_joint3', 'panda_joint4', 
    'panda_joint5', 'panda_joint6', 'panda_joint7'
]

kinDyn = KinDynComputations(model_path, joints_name_list)
# kinDyn.set_frame_velocity_representation(adam.Representations.BODY_FIXED_REPRESENTATION)
w_H_ee = kinDyn.forward_kinematics_fun("panda_grasptarget")
w_H_b = np.eye(4) # base frame 

q0 = jnp.array([0,-np.pi/3,0,-3, 0, 2.5, -0.7853])
args.update({'q0':q0})
# w_H_ee(w_H_b, q0)
for i, jt in enumerate(q0):
    robot[i] = jt
scene.render()

Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link0']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link1']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link2']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link4']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link5']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link6']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_link7']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_hand']/collision[1]
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_leftfinger']/collision[1]
Unknown tag "contact" in /robot[@name='panda']/link[@name='panda_leftfinger']
Unknown tag "material" in /robot[@name='panda']/link[@name='panda_rightfinger']/collision[1]
Unknown tag "contact" in /robot[@name='pan

In [6]:
def get_fk(q):
    return SE3.from_matrix(w_H_ee(w_H_b, q))

def RBF_kernel(x, xp, h=0.01):
    return jnp.exp(
        -jnp.sum((x-xp)**2)/h
    )

def camera_view_penalty(g, v, h=0.01):
    p = g.translation()
    rot = g.rotation()
    w = rot.apply(jnp.array([0.,0.,1.]))
    root, norm  = jnp.split(v, 2)
    # return jnp.exp(-jnp.sum((root-p)**2)/h - 0.01*jnp.sum((-norm-w)**2)/h)
    return jnp.exp(
        -jnp.sum((root-p)**2)/h #- 1e-1*jnp.sum((norm-w)**2)
    )*jnp.dot(-norm,w)




# def RBF_kernel(g, v, h=0.01):
#     p = g.translation()
#     rot = g.rotation()
#     w = rot.apply(jnp.array([0.,0.,1.]))
#     root, norm  = jnp.split(v, 2)
#     return jnp.exp(
#         -jnp.sum((root-p)**2)/h #- 1e-1*jnp.sum((norm-w)**2)
#     ) * jnp.exp(
#         -10*jnp.sum((-norm-w)**2)
#     )


def create_kernel_matrix(kernel):
    return vmap(vmap(kernel, in_axes=(0, None, None)), in_axes=(None, 0, None))

camera_view_matrix = create_kernel_matrix(camera_view_penalty)
KernelMatrix = create_kernel_matrix(RBF_kernel)

camera_view_matrix = create_kernel_matrix(camera_view_penalty)
KernelMatrix = create_kernel_matrix(RBF_kernel)
def emmd_loss(params, args):
    X = params['X']
    T = X.shape[0]
    h = args['h']
    q0 = args['q0']
    points    = args['points']
    norms     = args['normals']
    P_XI      = args['P_XI']

    g = vmap(get_fk)(X)
    p = g.translation()
    
    view_matrix = camera_view_matrix(g, jnp.hstack([points, norms]), 0.01)

    return np.sum(KernelMatrix(p, p, h))/(T**2) \
            - 2 * np.sum(P_XI @ KernelMatrix(p, points, h))/T \
            - 0.1*np.sum(P_XI@view_matrix)/T \
                    + 1e-3*jnp.sum(jnp.square(X[0]-q0)) + 1e-3*jnp.sum(jnp.square(X[-1]-q0))\
                        + 1e-4*jnp.sum(jnp.square(X[1:] - X[:-1]))


def eq_constr(params, args):
    # X = params['X']
    # q0 = args['q0']
    return jnp.zeros(1)

def ineq_constr(params, args):
    # X = params['X']
    # return jnp.square(X[1:] - X[:-1]) - 0.1**2
    return jnp.zeros(1)


In [7]:
T = 250
X = jnp.linspace(q0, q0+0.1, num=T)

params = {'X' : X}

# solver = AugmentedLagrangeSolver(params, emmd_loss, eq_constr, ineq_constr, args=args)
solver = AugmentedLagrangeSolver(
    params, emmd_loss, eq_constr, ineq_constr, 
    args=args)
# 

In [8]:
solver.solve(max_iter=15_000, eps=1e-4, alpha=1+1e-4)

0 0.2975314078372921
1 0.2662379480006152
2 0.12243916673196158
3 0.07269310178206786
4 0.04748996929758043
5 0.056051060975745456
6 0.03435245165925339
7 0.029081875788195547
8 0.02798563541132714
9 0.02100591182960817
10 0.02390949115412554
11 0.017858933998726435
12 0.025690685237469753
13 0.01802189648457104
14 0.019429142022848445
15 0.015349887371915843
16 0.014659104356625579
17 0.01271363726941853
18 0.012477358754422874
19 0.012671057582312881
20 0.013821995522601193
21 0.015586723604862386
22 0.014642948804850167
23 0.013237403917719957
24 0.013308408245738644
25 0.012503638564330975
26 0.011403019335550891
27 0.011711768512964385
28 0.009865114466214112
29 0.009783816535459634
30 0.01011669174690124
31 0.010748620211840603
32 0.010212240923897019
33 0.00915769733582313
34 0.00853012971067533
35 0.00756371194753025
36 0.00768506703681053
37 0.008029736814408248
38 0.008424991736041089
39 0.007843218181463133
40 0.007079997221241279
41 0.007401256957904814
42 0.006828765427370

## Animation

In [102]:
robot_img = []

sol = solver.solution
X = sol['X']
vertices = vmap(get_fk)(X).translation().astype(np.float32).T
search_path_vis = scene.vis['lines_segments']
search_path_obj = mc_geom.Line(mc_geom.PointsGeometry(vertices), mc_geom.MeshLambertMaterial(color=0x0000FF, linewidth=5))
search_path_vis.set_object(search_path_obj)
obj = Object(search_path_obj, name="lines_segments")
scene.add_object(obj)
for i, q in enumerate(X[::int(T/4)]):
    # _tf = np.array(get_fk(q).as_matrix()).astype(np.float64)
    # ee_frame.set_transform(_tf)
    _robot = Robot(name='robot{}'.format(i),urdf_path=PandaLoader().df_path, mesh_folder_path=Path(PandaLoader().model_path).parent.parent, opacity=0.75)
    robot_img.append(_robot)
    scene.add_robot(_robot)
    for j, jt in enumerate(q):
        _robot[j] = jt
# "Render the initial scene"
scene.render()

Object with the same name is already inside the scene, it will be replaced. 
Robot with the same name is already inside the scene, it will be replaced. 
Object with the same name is already inside the scene, it will be replaced. 
Object with the same name is already inside the scene, it will be replaced. 
Object with the same name is already inside the scene, it will be replaced. 
Object with the same name is already inside the scene, it will be replaced. 
Object with the same name is already inside the scene, it will be replaced. 
Object with the same name is already inside the scene, it will be replaced. 
Object with the same name is already inside the scene, it will be replaced. 
Object with the same name is already inside the scene, it will be replaced. 
Object with the same name is already inside the scene, it will be replaced. 
Object with the same name is already inside the scene, it will be replaced. 
Object with the same name is already inside the scene, it will be replaced. 


In [112]:
joint_trajectories = []
ee_trajectories = []
T = 250
X = jnp.linspace(q0, q0+0.1, num=T)

params = {'X' : X}

# solver = AugmentedLagrangeSolver(params, emmd_loss, eq_constr, ineq_constr, args=args)
solver = AugmentedLagrangeSolver(
    params, emmd_loss, eq_constr, ineq_constr, 
    args=args)

for _ in range(1000):
    solver.solve(max_iter=1, eps=1e-4, alpha=1+1e-4)
    sol = solver.solution
    X = sol['X']
    vertices = vmap(get_fk)(X).translation().astype(np.float32).T
    joint_trajectories.append(np.array(X))
    ee_trajectories.append(np.array(vertices))

0 0.2970158037314693
unsuccessful, tol:  0.2970158037314693
0 0.2663331735377961
unsuccessful, tol:  0.2663331735377961
0 0.12253623621181006
unsuccessful, tol:  0.12253623621181006
0 0.07234193796048619
unsuccessful, tol:  0.07234193796048619
0 0.047035696020338726
unsuccessful, tol:  0.047035696020338726
0 0.055396731669286714
unsuccessful, tol:  0.055396731669286714
0 0.034792982875632296
unsuccessful, tol:  0.034792982875632296
0 0.029063678267692024
unsuccessful, tol:  0.029063678267692024
0 0.026168442759899876
unsuccessful, tol:  0.026168442759899876
0 0.020380493409038886
unsuccessful, tol:  0.020380493409038886
0 0.019193744869358046
unsuccessful, tol:  0.019193744869358046
0 0.01988903663568285
unsuccessful, tol:  0.01988903663568285
0 0.015245959785575317
unsuccessful, tol:  0.015245959785575317
0 0.014980833052213415
unsuccessful, tol:  0.014980833052213415
0 0.01575246516407846
unsuccessful, tol:  0.01575246516407846
0 0.01403633926812135
unsuccessful, tol:  0.014036339268

In [129]:
obj.

False

In [130]:
scene.objects

{'robot0/panda_link0_0': <robomeshcat.object.Object at 0x2cdded0f0>,
 'robot0/panda_link1_0': <robomeshcat.object.Object at 0x2cddec7f0>,
 'robot0/panda_link2_0': <robomeshcat.object.Object at 0x2cddecfd0>,
 'robot0/panda_link3_0': <robomeshcat.object.Object at 0x2cde626e0>,
 'robot0/panda_link4_0': <robomeshcat.object.Object at 0x2cde60790>,
 'robot0/panda_link5_0': <robomeshcat.object.Object at 0x2cde613f0>,
 'robot0/panda_link6_0': <robomeshcat.object.Object at 0x2cddefdc0>,
 'robot0/panda_link7_0': <robomeshcat.object.Object at 0x2cde61240>,
 'robot0/panda_hand_0': <robomeshcat.object.Object at 0x2cdef6410>,
 'robot0/panda_leftfinger_0': <robomeshcat.object.Object at 0x2cddecaf0>,
 'robot0/panda_rightfinger_0': <robomeshcat.object.Object at 0x2cde60dc0>,
 'robot1/panda_link0_0': <robomeshcat.object.Object at 0x2cdd75750>,
 'robot1/panda_link1_0': <robomeshcat.object.Object at 0x2d07a78b0>,
 'robot1/panda_link2_0': <robomeshcat.object.Object at 0x2d07a6290>,
 'robot1/panda_link3_0':

In [155]:
anim = Animation(default_framerate=10)

for t in range(len(joint_trajectories[::10])):
    with anim.at_frame(scene.vis, t) as frame:
        # search_path_obj.geometry.position = ee_trajectories[t]

        scene["obj0"]._geometry.geometry.position = ee_trajectories[t]
        scene.vis["obj0"].set_object(scene["obj0"]._geometry, scene["obj0"]._material)
        # scene["obj0"]._set_object()
        # search_path_obj._geometry.geometry.position = ee_trajectories[t]
        # search_path_vis.set_object(search_path_obj)
        for i, q in enumerate(joint_trajectories[t][::int(T/4)]):
            # _tf = np.array(get_fk(q).as_matrix()).astype(np.float64)
            # ee_frame.set_transform(_tf)
            # _robot = Robot(name='robot{}'.format(i),urdf_path=PandaLoader().df_path, mesh_folder_path=Path(PandaLoader().model_path).parent.parent, opacity=0.75)
            for jnt_idx, _qi in enumerate(q):
                robot_img[i][jnt_idx] = _qi
        scene.render()

scene.vis.set_animation(anim)


In [141]:
# with scene.animation(fps=10):

#     for t in range(len(joint_trajectories[::10])):
#         # search_path_obj.geometry.position = ee_trajectories[t]

#         scene["obj0"]._geometry.geometry.position = ee_trajectories[t]
#         scene.vis["obj0"].set_object(scene["obj0"]._geometry, scene["obj0"]._material)
#         # scene["obj0"]._set_object()
#         # search_path_obj._geometry.geometry.position = ee_trajectories[t]
#         # search_path_vis.set_object(search_path_obj)
#         for i, q in enumerate(joint_trajectories[t][::int(T/4)]):
#             # _tf = np.array(get_fk(q).as_matrix()).astype(np.float64)
#             # ee_frame.set_transform(_tf)
#             # _robot = Robot(name='robot{}'.format(i),urdf_path=PandaLoader().df_path, mesh_folder_path=Path(PandaLoader().model_path).parent.parent, opacity=0.75)
#             for jnt_idx, _qi in enumerate(q):
#                 robot_img[i][jnt_idx] = _qi
#         scene.render()

## Stills

In [9]:
sol = solver.solution
X = sol['X']

vertices = vmap(get_fk)(X).translation().astype(np.float32).T

scene.vis['lines_segments'].set_object(mc_geom.Line(mc_geom.PointsGeometry(vertices), mc_geom.MeshLambertMaterial(color=0x0000FF, linewidth=2)))


# robot_img = []
# for i, q in enumerate(X[::int(T/4)]):
#     # _tf = np.array(get_fk(q).as_matrix()).astype(np.float64)
#     # ee_frame.set_transform(_tf)
#     _robot = Robot(name='robot{}'.format(i),urdf_path=PandaLoader().df_path, mesh_folder_path=Path(PandaLoader().model_path).parent.parent, opacity=0.75)
#     robot_img.append(_robot)
#     scene.add_robot(_robot)
#     for j, jt in enumerate(q):
#         _robot[j] = jt
# "Render the initial scene"
scene.render()

Robot with the same name is already inside the scene, it will be replaced. 
Object with the same name is already inside the scene, it will be replaced. 
Object with the same name is already inside the scene, it will be replaced. 
Object with the same name is already inside the scene, it will be replaced. 
Object with the same name is already inside the scene, it will be replaced. 
Object with the same name is already inside the scene, it will be replaced. 
Object with the same name is already inside the scene, it will be replaced. 
Object with the same name is already inside the scene, it will be replaced. 
Object with the same name is already inside the scene, it will be replaced. 
Object with the same name is already inside the scene, it will be replaced. 
Object with the same name is already inside the scene, it will be replaced. 
Object with the same name is already inside the scene, it will be replaced. 


In [ ]:
# # ps.set_up_dir("z_up")
# # ps.set_front_dir("neg_y_front")
# ps.init()


# ps_mesh = ps.register_surface_mesh("bunny", mesh.vertices, mesh.faces)
# # ps_points = ps.register_point_cloud("sampled points", args['points'][:,:3])
# # ps_mesh.add_scalar_quantity("face vals",vmap(info_distr, in_axes=(0,))(mesh.vertices))

# # ps_points.add_scalar_quantity("results", args['P_XI'])

# # ps_bunny_mesh.set_transparency(0.8)
# # ps_bunny_mesh.add_scalar_quantity('info_distr', mesh_func.func_vals, defined_on='vertices', cmap='blues')

# ps_traj         = ps.register_curve_network("trajectory", vmap(get_fk)(X).translation() , edges="line")


# for _ in range(1000):
#     solver.solve(eps=1e-5, max_iter=10)
#     sol = solver.solution
#     X = sol['X']

#     ps_traj.update_node_positions(vmap(get_fk)(X).translation() )
#     # ps_traj.add_vector_quantity("vec img", X[:,:3], enabled=True)
#     ps.frame_tick()

#     time.sleep(0.001)

In [ ]:
solver.solve(eps=1e-3)

In [22]:
sol = solver.solution

In [64]:
X = sol['X']

ps.init()

ps_bunny_mesh = ps.register_surface_mesh("bunny", mesh.vertices, mesh.faces)
# ps_bunny_mesh.set_transparency(0.8)
ps_bunny_mesh.add_scalar_quantity('info_distr', vmap(info_distr, in_axes=(0,))(mesh.vertices), defined_on='vertices', cmap='blues')

ps_traj         = ps.register_curve_network("trajectory", vmap(get_fk)(X).translation() , edges="line")

ps.show()

[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 4.1 Metal - 88
